In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential,Model
import cv2
from modules import * #Contain: stride, batch_size, nPosSoftSize, data_preparation(), decoder(), encoder()

2021-08-29 03:00:11.805601: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
encoder = encoder()
encoder.load_weights('weights/encoder_weights_best.h5')
decoder = decoder()
decoder.load_weights('weights/decoder_weights_best.h5')

2021-08-29 03:00:12.551848: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-29 03:00:12.552310: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-29 03:00:12.583161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 03:00:12.583466: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.68GHz coreCount: 34 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2021-08-29 03:00:12.583479: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-08-29 03:00:12.584790: I tensorflow/stream_executor/platform/defa

In [3]:
#Assemble EDNet 
x = encoder.output
for layer in decoder.layers[5:]:
    x = layer(x)
EDNet = tf.keras.Model(inputs=encoder.input, outputs=x)
EDNet.save_weights('weights/EDNet_weights.h5')
EDNet.summary()

In [5]:
def split(before):
    H = before.shape[0]
    W = before.shape[1]
    h = 128
    w = 128
    after = []
    for k1 in range(int(H / stride - 1)):
        for k2 in range(int(W / stride - 1)):
            after.append(before[k1 * stride:k1 * stride + h, k2 * stride:k2 * stride + w])
    after = np.array(after)
    return after

def save_res(name):
    '''
    name: Iput image path
    stride: Stride of the sliding window
    H,W: Height and witdh of the original images
    h,w: Height and witdh of the network inputs
    '''
    img = cv2.imread(name)/255
    img_ = split(img)
    res = EDNet.predict(img_)[:,:,:,0]
    res = np.round(res)
    
    
    H = img.shape[0]
    W = img.shape[1]
    h = 128
    w = 128    
    res_=np.zeros([H,W])
    #Assemble perdiction 
    for k1 in range(int((H-h)/ stride)+1):
        for k2 in range(int((W-w)/ stride)+1):
            res_[k1*stride:k1*stride+h,k2*stride:k2*stride+w]+=res[k1*(int((W-w)/ stride)+1)+k2]  
    res_=np.sign(res_)*255
    
    a=cv2.imwrite(name.replace('img','res'),res_)

name =  sorted(glob.glob(r'image/test/img/*'))
for i in range(len(name)):
    save_res(name[i])

2021-08-29 03:00:14.549984: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-29 03:00:14.568163: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3799900000 Hz
2021-08-29 03:00:14.932218: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-08-29 03:00:16.004830: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-08-29 03:00:16.009003: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 03:00:16.033209: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 03:00:16.056785: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 03:00